In [1]:
x=1

In [2]:

import scanpy as sc
import multigrate as mtg
import numpy as np

import tracemalloc
import scvi
import time
import torch

scvi.settings.seed = 0

Global seed set to 0
Global seed set to 0


In [3]:
torch.set_float32_matmul_precision('medium')

## cite 25

In [4]:
tracemalloc.start()

In [5]:
adata1 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/adata_gene_25.h5ad')
adata2 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/adata_protein_25.h5ad')

In [6]:
test_idx = adata2.obs['split'] == 'test'

adata1.X = adata1.layers['counts'].copy()

In [7]:
adata = mtg.data.organize_multimodal_anndatas(
        adatas=[[adata1], [adata2]],
        )

adata = adata[~test_idx].copy()

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [8]:
mtg.model.MultiVAE.setup_anndata(
    adata, 
    rna_indices_end=adata1.n_vars,
    categorical_covariate_keys=['Site'],
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [9]:
losses = ['nb', 'mse']
    
vae = mtg.model.MultiVAE(
    adata,
    losses=losses,
    modality_alignment='MMD',
    alignment_type='marginal',
)

In [10]:
adata

AnnData object with n_obs × n_vars = 22565 × 2134
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train', 'split', 'group', 'size_factors', '_scvi_batch'
    var: 'modality'
    uns: 'modality_lengths', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_extra_categorical_covs'
    layers: 'counts'

In [11]:
start_train_time = time.time()
vae.train(max_epochs=200)
train_time = time.time()-start_train_time

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskaya ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskay

Epoch 1/200:   0%|          | 0/200 [00:00<?, ?it/s]

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


Epoch 200/200: 100%|██████████| 200/200 [08:43<00:00,  3.06s/it, loss=563, v_num=1]   

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [08:43<00:00,  2.62s/it, loss=563, v_num=1]


In [12]:
current, peak = tracemalloc.get_traced_memory()
print(f"Training time: {train_time} seconds")
print(f"Peak training memory usage was {peak / (1024**2)} MB")
tracemalloc.reset_peak()

Training time: 527.5357298851013 seconds
Peak training memory usage was 1576.7288970947266 MB


## cite 50

In [13]:
tracemalloc.start()

In [14]:
adata1 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/adata_gene_50.h5ad')
adata2 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/adata_protein_50.h5ad')

In [15]:
test_idx = adata2.obs['split'] == 'test'

adata1.X = adata1.layers['counts'].copy()

In [16]:
adata = mtg.data.organize_multimodal_anndatas(
        adatas=[[adata1], [adata2]],
        )

adata = adata[~test_idx].copy()

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [17]:
mtg.model.MultiVAE.setup_anndata(
    adata, 
    rna_indices_end=adata1.n_vars,
    categorical_covariate_keys=['Site'],
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [18]:
losses = ['nb', 'mse']
    
vae = mtg.model.MultiVAE(
    adata,
    losses=losses,
    modality_alignment='MMD',
    alignment_type='marginal',
)

In [19]:
adata

AnnData object with n_obs × n_vars = 45130 × 2134
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train', 'split', 'group', 'size_factors', '_scvi_batch'
    var: 'modality'
    uns: 'modality_lengths', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_extra_categorical_covs'
    layers: 'counts'

In [20]:
start_train_time = time.time()
vae.train(max_epochs=200)
train_time = time.time()-start_train_time

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskaya ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskay

Epoch 200/200: 100%|██████████| 200/200 [16:54<00:00,  5.13s/it, loss=556, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [16:54<00:00,  5.07s/it, loss=556, v_num=1]


In [21]:
current, peak = tracemalloc.get_traced_memory()
print(f"Training time: {train_time} seconds")
print(f"Peak training memory usage was {peak / (1024**2)}MB")
tracemalloc.reset_peak()

Training time: 1014.945752620697 seconds
Peak training memory usage was 1685.4433517456055MB


## cite 75

In [22]:
tracemalloc.start()

In [23]:
adata1 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/adata_gene_75.h5ad')
adata2 = sc.read('/lustre/groups/ml01/code/anastasia.litinetskaya/multigrate_reproducibility/imputation/pipeline/data/input/cite/adata_protein_75.h5ad')

In [24]:
test_idx = adata2.obs['split'] == 'test'

adata1.X = adata1.layers['counts'].copy()

In [25]:
adata = mtg.data.organize_multimodal_anndatas(
        adatas=[[adata1], [adata2]],
        )

adata = adata[~test_idx].copy()

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [26]:
tracemalloc.start()

In [27]:
mtg.model.MultiVAE.setup_anndata(
    adata, 
    rna_indices_end=adata1.n_vars,
    categorical_covariate_keys=['Site'],
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [28]:
losses = ['nb', 'mse']
    
vae = mtg.model.MultiVAE(
    adata,
    losses=losses,
    modality_alignment='MMD',
    alignment_type='marginal',
)

In [29]:
adata

AnnData object with n_obs × n_vars = 67695 × 2134
    obs: 'GEX_n_genes_by_counts', 'GEX_pct_counts_mt', 'GEX_size_factors', 'GEX_phase', 'ADT_n_antibodies_by_counts', 'ADT_total_counts', 'ADT_iso_count', 'cell_type', 'batch', 'ADT_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'is_train', 'split', 'group', 'size_factors', '_scvi_batch'
    var: 'modality'
    uns: 'modality_lengths', '_scvi_uuid', '_scvi_manager_uuid'
    obsm: '_scvi_extra_categorical_covs'
    layers: 'counts'

In [30]:
start_train_time = time.time()
vae.train(max_epochs=200)
train_time = time.time()-start_train_time

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskaya ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multigrate/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /lustre/groups/ml01/workspace/anastasia.litinetskay

Epoch 200/200: 100%|██████████| 200/200 [24:14<00:00,  6.79s/it, loss=559, v_num=1]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 200/200: 100%|██████████| 200/200 [24:14<00:00,  7.27s/it, loss=559, v_num=1]


In [31]:
current, peak = tracemalloc.get_traced_memory()
print(f"Training time: {train_time} seconds")
print(f"Peak training memory usage was {peak / (1024**2)}MB")
tracemalloc.reset_peak()

Training time: 1455.502221107483 seconds
Peak training memory usage was 2229.807662963867MB
